<a href="https://colab.research.google.com/github/Rahafsjob/Capstone_Project_WeCloudData/blob/main/Stage2/Copy_of_rawData_Final_Create_Tables_and_DataProfiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries

In [ ]:
!pip install SQLAlchemy

In [ ]:
!pip install PyMySQL

# Make Connection

In [ ]:
from sqlalchemy import create_engine

ENDPOINT = "database-1.cipmeg2oi9nm.us-east-1.rds.amazonaws.com"
PORT = "3306"
USERNAME = "admin"
PASSWORD = "Rahaf123_"
DATABASENAME = "demo_db"

engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASENAME}")

# Test Connection

In [ ]:
from sqlalchemy import text

with engine.connect() as connection:
  result = connection.execute(text("SHOW TABLES"))
  tables = result.fetchall()
  print(tables)

# Create Table

## Attendance Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS attendance"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS attendance (
	  StudentID VARCHAR(15),
    attendance_date VARCHAR(70),
    attendance_data VARCHAR(5),
    course_name VARCHAR(200),
    cohort_number VARCHAR(5),
    filename VARCHAR(200),
    Final_Cohort VARCHAR(100),
    level VARCHAR(5)
  )
  """
  ))
  connection.commit()

## Assiements Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS assiements"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS assiements (
	  Student_ID VARCHAR(15),
    week1 VARCHAR(20),
    week2 VARCHAR(20),
    week3 VARCHAR(20),
    week4 VARCHAR(20),
    week5 VARCHAR(20),
    week6 VARCHAR(20),
    week7 VARCHAR(20),
    final_score float(5),
    name VARCHAR(200)
  )
  """
  ))
  connection.commit()


## Quizzes Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS Quizzes"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS Quizzes (
	  Student_ID VARCHAR(15),
    Quiz1 float(20),
    Quiz2 float(20),
    Quiz3 float(20),
    Quiz4 float(20),
    Quiz5 float(20),
    Quiz6 float(20),
    Quiz7 float(20),
    Quiz8 float(20),
    name VARCHAR(300)
  )
  """
  ))
  connection.commit()


## Centers Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS Centers"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS Centers (
	  CenterID VARCHAR(10) NOT NULL
  )
  """
  ))
  connection.commit()

In [ ]:
import pandas as pd # Import the pandas library and assign it to the alias 'pd'


In [ ]:
centers_df = pd.read_csv("/content/Centers.csv")
centers_df.head()

In [ ]:
import numpy as np
df = centers_df.replace({np.nan: None})

In [ ]:
with engine.connect() as connection:
  for index, line in centers_df.iterrows():
    # Use parameterized queries to avoid SQL injection vulnerabilities and handle quoting issues
    query = text("""
        INSERT INTO Centers (CenterID)
        VALUES (:CenterID)
    """)
    values = {
        'CenterID': line['Center ID']
    }
    connection.execute(query, values)
  connection.commit()

In [ ]:
query = f"select * from Centers"
centers_df=pd.read_sql(query, engine)
centers_df.head()

## Cohort_Schedule Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS Cohort_Schedule"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS Cohort_Schedule (
	  level INT,
    name VARCHAR(255),
    cohort VARCHAR(50),
    start_date VARCHAR(100),
    end_date VARCHAR(100),
    dependency VARCHAR(255),
    cohort_full_info VARCHAR(255),
    start_date_dateformat VARCHAR(100),
    end_date_dateformat VARCHAR(100)
  )
  """
  ))
  connection.commit()

In [ ]:
cohort_schedule_df = pd.read_csv("/content/Cohort Schedule.tsv", sep="\t")
cohort_schedule_df.head()

In [ ]:
import numpy as np
cohort_schedule_df = cohort_schedule_df.replace({np.nan: None})

In [ ]:
with engine.connect() as connection:
    for index, line in cohort_schedule_df.iterrows():
        query = text("""
            INSERT INTO Cohort_Schedule (
                level, name, cohort, start_date, end_date, dependency,
                cohort_full_info, start_date_dateformat, end_date_dateformat
            )
            VALUES (
                :level, :name, :cohort, :start_date, :end_date, :dependency,
                :cohort_full_info, :start_date_dateformat, :end_date_dateformat
            )
        """)
        values = {
            'level': line['level'],
            'name': line['name'],
            'cohort': line['cohort'],
            'start_date': line['start_date'],
            'end_date': line['end_date'],
            'dependency': line['dependency'],
            'cohort_full_info': line['cohort_full_info'],
            'start_date_dateformat': line['start_date_dateformat'],
            'end_date_dateformat': line['end_date_dateformat']
        }
        connection.execute(query, values)
    connection.commit()


## Student_Status Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS Student_Status"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS Student_Status (
	  StudentID VARCHAR(10) NOT NULL,
    CenterID VARCHAR(10) NOT NULL,
    target_level VARCHAR(10)
  )
  """
  ))
  connection.commit()

In [ ]:
student_status_df = pd.read_csv("/content/Student Status.csv")
student_status_df.head()

In [ ]:
import numpy as np
student_status_df = student_status_df.replace({np.nan: None})

In [ ]:
with engine.connect() as connection:
    for index, line in student_status_df.iterrows():
        query = text("""
            INSERT INTO Student_Status (StudentID, CenterID, target_level)
            VALUES (:StudentID, :CenterID, :target_level)
        """)
        values = {
            'StudentID': line['Student ID'],
            'CenterID': line['Center ID'],
              'target_level': line['Target Level']
        }
        connection.execute(query, values)
    connection.commit()

## Change_Requests Table

In [ ]:
with engine.connect() as connection:
    connection.execute(text("DROP TABLE IF EXISTS Change_Requests"))
    connection.execute(text("""
        CREATE TABLE IF NOT EXISTS Change_Requests (
            studentID VARCHAR(20),
            centerID VARCHAR(20),
            level FLOAT,
            name VARCHAR(100),
            cohort_affected VARCHAR(20),
            request_date VARCHAR(50),
            request VARCHAR(100),
            reschedule_to VARCHAR(20),
            withdraw_reason VARCHAR(300)
        )
    """))
    connection.commit()

In [ ]:
change_requests_df = pd.read_csv("/content/Change Requests.tsv", sep="\t")
change_requests_df.head()

In [ ]:
change_requests_df = change_requests_df.replace({np.nan: None})

In [ ]:
change_requests_df.columns

In [ ]:
with engine.connect() as connection:
    for index, line in change_requests_df.iterrows():
        query = text("""
            INSERT INTO Change_Requests (
                studentID, centerID, level, name, cohort_affected, request_date,
                request, reschedule_to, withdraw_reason
            )
            VALUES (
                :studentID, :centerID, :level, :name, :cohort_affected, :request_date, :request, :reschedule_to,
                :withdraw_reason
            )
        """)
        values = {
            'studentID': line['studentID'],
            'centerID': line['centerID'],
            'level': line['level'],
            'name': line['name'],
            'cohort_affected': line['cohort_affected'],
            'request_date': line['request_date'],
            'request': line['request'],
            'reschedule_to': line['reschedule_to'],
            'withdraw_reason': line['withdraw_reason']
        }
        connection.execute(query, values)
    connection.commit()

## Change_New_Joiners

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS Change_New_Joiners"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS Change_New_Joiners (
	  StudentID VARCHAR(10),
    CenterID VARCHAR(10),
    title VARCHAR(100),
    DateAdded VARCHAR(100),
    Type VARCHAR(100)
  )
  """
  ))
  connection.commit()

In [ ]:
Change_New_Joiners_df = pd.read_csv("/content/Change New Joiners.csv")
Change_New_Joiners_df.head()

In [ ]:
Change_New_Joiners_df.columns

In [ ]:
Change_New_Joiners_df = Change_New_Joiners_df.replace({np.nan: None})

In [ ]:
with engine.connect() as connection:
    for index, line in Change_New_Joiners_df.iterrows():
        query = text("""
            INSERT INTO Change_New_Joiners (
              StudentID, CenterID, title, DateAdded, Type
            )
            VALUES (
              :StudentID, :CenterID, :title, :DateAdded, :Type
            )
        """)
        values = {
            'StudentID': line['Student ID'],
            'CenterID': line['Center ID'],
            'title': line['title'],
            'DateAdded': line['Date Added'],
            'Type': line['Type']
        }
        connection.execute(query, values)
    connection.commit()

## Cohort_Assignment Table

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS Cohort_Assignment"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS Cohort_Assignment (
	  studentID VARCHAR(50),
    centerID VARCHAR(50),
    level VARCHAR(50),
    name VARCHAR(100),
    cohort VARCHAR(50),
    start_date VARCHAR(100),
    end_date VARCHAR(100),
    cohort_schedule VARCHAR(100),
    cohort_status VARCHAR(50),
    enrolment_confirmation VARCHAR(50),
    withdraw_reason1 VARCHAR(100),
    withdraw_reason2 VARCHAR(100),
    level_status_calculated VARCHAR(50),
    new_joiner VARCHAR(50),
    level_graduation_indicator VARCHAR(50),
    attendance_rate VARCHAR(50),
    quiz_complete VARCHAR(50),
    project_submit VARCHAR(50)
  )
  """
  ))
  connection.commit()

In [ ]:
Cohort_Assignment_df = pd.read_csv("/content/Cohort Assignment.tsv", sep="\t")
Cohort_Assignment_df.head()

In [ ]:
Cohort_Assignment_df = Cohort_Assignment_df.replace({np.nan: None})

In [ ]:
with engine.connect() as connection:
    for index, line in Cohort_Assignment_df.iterrows():
        query = text("""
            INSERT INTO Cohort_Assignment (
              studentID, centerID, level, name, cohort, start_date, end_date,
              cohort_schedule, cohort_status, enrolment_confirmation, withdraw_reason1,
              withdraw_reason2, level_status_calculated, new_joiner, level_graduation_indicator,
              attendance_rate, quiz_complete, project_submit
            )
            VALUES (
              :studentID, :centerID, :level, :name, :cohort, :start_date, :end_date,
              :cohort_schedule, :cohort_status, :enrolment_confirmation, :withdraw_reason1,
              :withdraw_reason2, :level_status_calculated, :new_joiner, :level_graduation_indicator,
              :attendance_rate, :quiz_complete, :project_submit
            )
        """)
        values = {
            'studentID': line['studentID'],
            'centerID': line['centerID'],
            'level': line['level'],
            'name': line['name'],
            'cohort': line['cohort'],
            'start_date': line['start'],
            'end_date': line['end'],
            'cohort_schedule': line['cohort_schedule'],
            'cohort_status': line['cohort_status'],
            'enrolment_confirmation': line['enrolment_confirmation'],
            'withdraw_reason1': line['withdraw_reason1'],
            'withdraw_reason2': line['withdraw_reason2'],
            'level_status_calculated': line['level_status_calculated'],
            'new_joiner': line['new_joiner'],
            'level_graduation_indicator': line['level_graduation_indicator'],
            'attendance_rate': line['attendance_rate'],
            'quiz_complete': line['quiz_complete'],
            'project_submit': line['project_submit']
        }
        connection.execute(query, values)
    connection.commit()

# Read Data

In [ ]:
import pandas as pd

## Attendance Table

In [ ]:
df = pd.read_csv("/content/final_attendance_Stage2_with_levels.csv")
df.head()

In [ ]:
import numpy as np
df = df.replace({np.nan: None})

In [ ]:
for _, e in df.iterrows():
  print(e['StudentID'], e['attendance_date'], e["attendance_data"],
        e["course_name"], e["cohort_number"], e["filename"], e["Final_Cohort"], e["level"])

## Assiements Table

In [ ]:
df_assiements = pd.read_csv("/content/assiements.csv")
df_assiements.head()

In [ ]:
for _, e in df_assiements.iterrows():
  print(e['Student_ID'], e['Week1'], e['Week2'], e['Week3'],
        e['Week4'], e['Week5'], e['Week6'], e['Week7'], e['Final Score'], e["name"],)

## Quizzes Table

In [ ]:
df_Quizzes = pd.read_csv("/content/Quizzes.csv")
df_Quizzes.head()

In [ ]:
import numpy as np
df_Quizzes = df_Quizzes.replace({'Null': None})

In [ ]:
for _, e in df_Quizzes.iterrows():
  print(e['Student_ID'], e['Quiz1'], e['Quiz2'], e['Quiz3'],
        e['Quiz4'], e['Quiz5'], e['Quiz6'], e['Quiz7'], e['Quiz8'], e["name"],)

# Insert data into table

## Attendance Table

In [ ]:
with engine.connect() as connection:
  for index, line in df.iterrows():
    # Use parameterized queries to avoid SQL injection vulnerabilities and handle quoting issues
    query = text("""
        INSERT INTO attendance (StudentID, attendance_date, attendance_data, course_name, cohort_number, filename, Final_Cohort, level)
        VALUES (:StudentID, :attendance_date, :attendance_data, :course_name, :cohort_number, :filename , :Final_Cohort, :level)
    """)
    # Pass values as a dictionary to the execute method
    # Replace NaN values with None before inserting into the database
    values = {
        'StudentID': line['StudentID'],
        'attendance_date': line['attendance_date'],
        'attendance_data': line['attendance_data'],
        'course_name': line['course_name'],
        'cohort_number': line['cohort_number'],
        'filename': line['filename'],
        'Final_Cohort': line['Final_Cohort'],
        'level': line['level']
    }
    connection.execute(query, values)
  connection.commit()

## Assiements Table

In [ ]:
with engine.connect() as connection:
  for index, line in df_assiements.iterrows():
    # Use parameterized queries to avoid SQL injection vulnerabilities and handle quoting issues
    query = text("""
        INSERT INTO assiements (Student_ID, week1, week2, week3, week4, week5, week6, week7, final_score, name)
        VALUES (:Student_ID, :week1, :week2, :week3, :week4, :week5, :week6, :week7, :final_score, :name)
    """)
    # Pass values as a dictionary to the execute method
    # Replace NaN values with None before inserting into the database
    values = {
        'Student_ID': line['Student_ID'],
        'week1': line['Week1'],
        'week2': line['Week2'],
        'week3': line['Week3'],
        'week4': line['Week4'],
        'week5': line['Week5'],
        'week6': line['Week6'],
        'week7': line['Week7'],
        'final_score': line['Final Score'],
        'name': line['name']
    }
    connection.execute(query, values)
  connection.commit()

## Quizzes Table

In [ ]:
with engine.connect() as connection:
  for index, line in df_Quizzes.iterrows():
    # Use parameterized queries to avoid SQL injection vulnerabilities and handle quoting issues
    query = text("""
        INSERT INTO Quizzes (Student_ID, Quiz1, Quiz2, Quiz3, Quiz4, Quiz5, Quiz6, Quiz7, Quiz8, name)
        VALUES (:Student_ID, :Quiz1, :Quiz2, :Quiz3, :Quiz4, :Quiz5, :Quiz6, :Quiz7, :Quiz8, :name)
    """)
    # Pass values as a dictionary to the execute method
    # Replace NaN values with None before inserting into the database
    values = {
        'Student_ID': line['Student_ID'],
        'Quiz1': line['Quiz1'],
        'Quiz2': line['Quiz2'],
        'Quiz3': line['Quiz3'],
        'Quiz4': line['Quiz4'],
        'Quiz5': line['Quiz5'],
        'Quiz6': line['Quiz6'],
        'Quiz7': line['Quiz7'],
        'Quiz8': line['Quiz8'],
        'name': line['name']
    }
    connection.execute(query, values)
  connection.commit()

# DATAGRIP TABLES

### attempts

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS attempts"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS attempts (
	  id INT,
    score double,
    timer double,
    trial varchar(1000),
    exam_id varchar (1000),
    user_id int,
    duration double,
    cohort_id varchar(1000),
    pauseable varchar(1000),
    attempt_id varchar(1000),
    created_at datetime,
    pool_count int,
    material_id int,
    resource_id varchar(1000),
    question_ids varchar(1000),
    submitted_at datetime,
    choices_count int,
    correct_count double,
    cohort_course_id int,
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### chapters

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS chapters"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS chapters (
	  id int,
    name varchar(1000),
    handle varchar(1000),
    exam_id varchar(1000),
    position int,
    published varchar(1000),
    created_at datetime,
    updated_at datetime,
    description varchar(1000),
    resource_id varchar(1000),
    cohort_course_id int,
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### choices

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS choices"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS choices (
    id int,
    grade double,
    notes varchar(1000),
    choice varchar(1000),
    graded varchar(1000),
    comment varchar(1000),
    correct varchar(1000),
    user_id int,
    attempt_id int,
    question_id int,
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### cohort_course_enrollments

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS cohort_course_enrollments"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS cohort_course_enrollments (
	  id int,
    status varchar(1000),
    exam_id varchar(1000),
    user_id int,
    created_at datetime,
    cohort_course_id int,
    client_id varchar(1000)
    )
  """
    ))
  connection.commit()

### cohort_courses

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS cohort_courses"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS cohort_courses (
	  id int,
    name varchar(1000),
    weight varchar(1000),
    exam_id int,
    cohort_id int,
    published varchar(1000),
    created_at datetime,
    resource_id varchar(1000),
    description varchar(1000),
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### cohort_enrollments

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS cohort_enrollments"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS cohort_enrollments (
	  id int,
    user_id int,
    cohort_id int,
    created_at datetime,
    portfolio_published varchar(1000),
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### cohorts

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS cohorts"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS cohorts (
	  id int,
    days varchar(1000),
    name varchar(1000),
    times varchar(1000),
    end_date date,
    start_date date,
    program_id int,
    created_at datetime,
    resource_id varchar(1000),
    cohort_enrollments_count int,
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### handin_attachments

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS handin_attachments"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS handin_attachments (
	  id int,
    s3_key varchar(1000),
    user_id int,
    filename varchar(1000),
    created_at datetime,
    user_material_id int,
    client_id varchar(1000)
  )
  """
    ))
  connection.commit()

### materials

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS materials"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS materials (
	  id INT ,
    kind VARCHAR(1000),
    name VARCHAR(1000),
    trial VARCHAR(1000),
    handle VARCHAR(1000),
    exam_id VARCHAR(1000),
    due_date DATETIME,
    duration DOUBLE,
    position INT,
    topic_id INT,
    chapter_id DOUBLE,
    created_at DATETIME,
    cohort_course_id VARCHAR(1000),
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### program_courses

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS program_courses"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS program_courses (
    id INT,
    exam_id INT,
    position DOUBLE,
    program_id INT,
    resource_id VARCHAR(1000),
    created_at DATETIME,
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### programs

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS programs"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS programs (
    id INT,
    kind VARCHAR(1000),
    name VARCHAR(1000),
    handle VARCHAR(1000),
    published VARCHAR(1000),
    description VARCHAR(1000),
    resource_id VARCHAR(1000),
    cohorts_count INT,
    free_cohort_id DOUBLE,
    organization_id DOUBLE,
    program_category_id VARCHAR(1000),
    program_courses_count INT,
    created_at DATETIME,
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### questions

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS questions"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS questions (
    id INT ,
    kind VARCHAR(1000) ,
    grade DOUBLE ,
    trial VARCHAR(1000),
    exam_id VARCHAR(1000),
    question VARCHAR(1000),
    domain_id VARCHAR(1000),
    published VARCHAR(1000),
    material_id INT,
    created_at DATETIME,
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### topics

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS topics"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS topics (
    id INT,
    name VARCHAR(1000),
    exam_id VARCHAR(1000),
    position INT,
    published VARCHAR(1000),
    chapter_id INT,
    resource_id VARCHAR(1000),
    cohort_course_id DOUBLE,
    created_at DATETIME,
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### user_exams

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS user_exams"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS user_exams (
    id INT,
    exam_id VARCHAR(1000),
    user_id INT,
    cohort_course_id INT,
    stars_count DOUBLE,
    stars_total_count DOUBLE,
    created_at DATETIME,
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### user_materials

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS user_materials"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS user_materials (
    id INT,
    late VARCHAR(1000),
    status VARCHAR(1000),
    flashed VARCHAR(1000),
    user_id INT,
    marked_at VARCHAR(1000),
    marking_admin_id DOUBLE,
    material_id INT,
    handin_attachment_id DOUBLE,
    cohort_course_enrollment_id INT,
    quiz_results VARCHAR(1000),
    quized VARCHAR(1000),
    quized_at DATETIME,
    user_checked VARCHAR(1000),
    grading_notes VARCHAR(1000),
    grading_score DOUBLE ,
    grading_total VARCHAR(1000),
    client_id VARCHAR(1000)
  )
  """
    ))
  connection.commit()

### users

In [ ]:
with engine.connect() as connection:
  connection.execute(text("DROP TABLE IF EXISTS users"))
  connection.execute(text("""
  CREATE TABLE IF NOT EXISTS users (
    student_id  VARCHAR(1000),
    role  VARCHAR(1000),
    client_id VARCHAR(1000),
    id INT,
    first_name VARCHAR(1000) , -- ?
    last_name VARCHAR(1000) , -- ?
    email VARCHAR(1000)
  )
  """
    ))
  connection.commit()

# DATA PROFILING (BEFORE)

In [ ]:
!pip install ydata-profiling pymysql

In [ ]:
!pip install pandasql

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from ydata_profiling import ProfileReport
from pandasql import sqldf

### attendance

In [ ]:
query = f"select * from attendance"
dfataFrame1=pd.read_sql(query, engine)
dfataFrame1.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame1, title="attendance Report")
customer_profile.to_file("attendance_report.html")
customer_profile.to_notebook_iframe()

### assiements

In [ ]:
query = f"select * from assiements"
dfataFrame2=pd.read_sql(query, engine)
dfataFrame2.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame2, title="assiements Report")
customer_profile.to_file("assiements_report.html")
customer_profile.to_notebook_iframe()

### Quizzes

In [ ]:
query = f"select * from Quizzes"
dfataFrame3=pd.read_sql(query, engine)
dfataFrame3.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame3, title="Quizzes Report")
customer_profile.to_file("Quizzes_report.html")
customer_profile.to_notebook_iframe()

### attempts

In [ ]:
query = f"select * from attempts"
dfataFrame4=pd.read_sql(query, engine)
dfataFrame4.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame4, title="attempts Report")
customer_profile.to_file("attempts_report.html")
customer_profile.to_notebook_iframe()

### chapters

In [ ]:
query = f"select * from chapters"
dfataFrame5=pd.read_sql(query, engine)
dfataFrame5.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame5, title="chapters Report")
customer_profile.to_file("chapters_report.html")
customer_profile.to_notebook_iframe()

### cohort_course_enrollments

In [ ]:
query = f"select * from cohort_course_enrollments"
dfataFrame6=pd.read_sql(query, engine)
dfataFrame6.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame6, title="cohort_course_enrollments Report")
customer_profile.to_file("cohort_course_enrollments_report.html")
customer_profile.to_notebook_iframe()

### cohort_courses

In [ ]:
query = f"select * from cohort_courses"
dfataFrame7=pd.read_sql(query, engine)
dfataFrame7.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame7, title="cohort_courses Report")
customer_profile.to_file("cohort_courses.html")
customer_profile.to_notebook_iframe()

### cohort_enrollments

In [ ]:
query = f"select * from cohort_enrollments"
dfataFrame8=pd.read_sql(query, engine)
dfataFrame8.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame8, title="cohort_enrollments Report")
customer_profile.to_file("cohort_enrollments.html")
customer_profile.to_notebook_iframe()

### cohorts

In [ ]:
query = f"select * from cohorts"
dfataFrame9=pd.read_sql(query, engine)
dfataFrame9.head()

In [ ]:
dfataFrame9.shape

In [ ]:
customer_profile= ProfileReport(dfataFrame9, title="cohorts Report")
customer_profile.to_file("cohorts.html")
customer_profile.to_notebook_iframe()

### handin_attachments

In [ ]:
query = f"select * from handin_attachments"
dfataFrame10=pd.read_sql(query, engine)
dfataFrame10.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame10, title="handin_attachments Report")
customer_profile.to_file("handin_attachments.html")
customer_profile.to_notebook_iframe()

### materials

In [ ]:
query = f"select * from materials"
dfataFrame11=pd.read_sql(query, engine)
dfataFrame11.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame11, title="materials Report")
customer_profile.to_file("materials.html")
customer_profile.to_notebook_iframe()

### program_courses

In [ ]:
query = f"select * from program_courses"
dfataFrame12=pd.read_sql(query, engine)
dfataFrame12.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame12, title="program_courses Report")
customer_profile.to_file("program_courses.html")
customer_profile.to_notebook_iframe()

### programs

In [ ]:
query = f"select * from programs"
dfataFrame13=pd.read_sql(query, engine)
dfataFrame13.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame13, title="programs Report")
customer_profile.to_file("programs.html")
customer_profile.to_notebook_iframe()

### questions

In [ ]:
query = f"select * from questions"
dfataFrame14=pd.read_sql(query, engine)
dfataFrame14.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame14, title="questions Report")
customer_profile.to_file("questions.html")
customer_profile.to_notebook_iframe()

### topics

In [ ]:
query = f"select * from topics"
dfataFrame15=pd.read_sql(query, engine)
dfataFrame15.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame15, title="topics Report")
customer_profile.to_file("topics.html")
customer_profile.to_notebook_iframe()

### user_exams

In [ ]:
query = f"select * from user_exams"
dfataFrame16=pd.read_sql(query, engine)
dfataFrame16.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame16, title="user_exams Report")
customer_profile.to_file("user_exams.html")
customer_profile.to_notebook_iframe()

### user_materials

In [ ]:
query = f"select * from user_materials"
dfataFrame17=pd.read_sql(query, engine)
dfataFrame17.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame17, title="user_materials Report")
customer_profile.to_file("user_materials.html")
customer_profile.to_notebook_iframe()

### users

In [ ]:
query = f"select * from users"
dfataFrame18=pd.read_sql(query, engine)
dfataFrame18.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame18, title="users Report")
customer_profile.to_file("users.html")
customer_profile.to_notebook_iframe()

### Centers

In [ ]:
query = f"select * from Centers"
dfataFrame19=pd.read_sql(query, engine)
dfataFrame19.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame19, title="Centers report")
customer_profile.to_file("Centers.html")
customer_profile.to_notebook_iframe()

### Cohort_Schedule

In [ ]:
query = f"select * from Cohort_Schedule"
dfataFrame20=pd.read_sql(query, engine)
dfataFrame20.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame20, title="Cohort Schedule Report")
customer_profile.to_file("Cohort Schedule.html")
customer_profile.to_notebook_iframe()

### Student_Status

In [ ]:
query = f"select * from Student_Status"
dfataFrame21=pd.read_sql(query, engine)
dfataFrame21.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame21, title="Student_Status Report")
customer_profile.to_file("Student Status.html")
customer_profile.to_notebook_iframe()

### Change_Requests

In [ ]:
query = f"select * from Change_Requests"
dfataFrame22=pd.read_sql(query, engine)
dfataFrame22.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame22, title="Change_Requests Report")
customer_profile.to_file("Change Requests.html")
customer_profile.to_notebook_iframe()

### Change_New_Joiners

In [ ]:
query = f"select * from Change_New_Joiners"
dfataFrame23=pd.read_sql(query, engine)
dfataFrame23.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame23, title="Change_New_Joiners Report")
customer_profile.to_file("ChangeNewJoiners.html")
customer_profile.to_notebook_iframe()

### Cohort_Assignment

In [ ]:
query = f"select * from Cohort_Assignment"
dfataFrame24=pd.read_sql(query, engine)
dfataFrame24.head()

In [ ]:
customer_profile= ProfileReport(dfataFrame24, title="Cohort_Assignment Report")
customer_profile.to_file("Cohort_Assignment.html")
customer_profile.to_notebook_iframe()